In [ ]:
### adding a change

In [ ]:
import teehr.queries.duckdb as tqd

import dashboard_utils as du
import importlib
from datetime import datetime, timedelta
from pathlib import Path
import pandas as pd
import geopandas as gpd

In [ ]:
# evaluation study directory
STUDY_DIR = Path("/home", "jovyan", "shared", "rti-eval", "post-event-example")

# medium range streamflow forecast evaluation files 
scenario = dict(
    scenario_name="medium_range",
    variable="streamflow",
    primary_filepath=Path(STUDY_DIR, "timeseries", "usgs", "*.parquet"),
    secondary_filepath=Path(STUDY_DIR, "timeseries", "medium_range_mem1", "*.parquet"),
    crosswalk_filepath=Path(STUDY_DIR, "geo", "usgs_nwm22_crosswalk.parquet"),
    geometry_filepath=Path(STUDY_DIR, "geo", "usgs_geometry.parquet")
)
attribute_paths = dict(
    usgs_upstream_area=Path(STUDY_DIR, "geo", "usgs_attr_upstream_area.parquet"),
    usgs_ecoregions=Path(STUDY_DIR, "geo", "usgs_attr_ecoregions.parquet"),
    usgs_stream_order=Path(STUDY_DIR, "geo", "usgs_attr_stream_order.parquet"),
    usgs_huc_crosswalk=Path(STUDY_DIR, "geo", "usgs_huc12_crosswalk.parquet"),
)



In [ ]:
# build the filters
filters=[]    
location_list = du.get_usgs_locations_within_huc(huc_level=2, huc_id='18', attribute_paths=attribute_paths)
location_list.sort()
filters.append(
    {
        "column": "primary_location_id",
        "operator": "in",
        "value": location_list
    }
)
filters.append(
    {
        "column": "reference_time",
        "operator": "=",
        "value": f"{datetime(2023, 1, 1, 12)}"
    }
)
# get metrics
gdf = tqd.get_metrics(
    primary_filepath=scenario["primary_filepath"],
    secondary_filepath=scenario["secondary_filepath"],
    crosswalk_filepath=scenario["crosswalk_filepath"],       
    group_by=['primary_location_id','reference_time'],    
    order_by=['primary_location_id'],
    filters=filters,
    return_query=False,
    geometry_filepath=scenario["geometry_filepath"],       
    include_geometry=True,
    include_metrics=['max_value_delta'],
) 

print(len(gdf))
print(gdf.iloc[220:230,0:3])

In [ ]:
order_by=['primary_location_id']
gdf = gdf.sort_values(order_by)
print(gdf.iloc[220:230,0:3])